In [1]:
# Paso 0: Instalación (si estás en un entorno local)
# !pip install pytorch-lightning torchvision

### 📌 Paso 1: Importar módulos necesarios
Importamos PyTorch Lightning y las herramientas necesarias para crear el modelo, cargar datos y entrenar.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pytorch_lightning as pl

C:\Users\PC MASTER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 📚 Paso 2: Dataset y DataLoader
Utilizamos MNIST con `torchvision.datasets`, aplicando una transformación para convertir imágenes en tensores.

In [3]:
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:08<00:00, 1.22MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 375kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 1.69MB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 9.03MB/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



### ⚡ Paso 3: Definir un modelo con PyTorch Lightning
Extendemos `pl.LightningModule`, que organiza automáticamente el ciclo de entrenamiento, validación y optimización.

In [4]:
class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

### 🔁 Paso 4: Entrenamiento con Trainer
El objeto `Trainer` se encarga de entrenar automáticamente el modelo Lightning con muchas opciones configurables.

In [7]:
model = LitModel()
trainer = pl.Trainer(max_epochs=1, enable_progress_bar=True)
trainer.fit(model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


FailedPreconditionError: C:\Users\PC MASTER\desktop\ucreativa-ai\07-FUNDAMENTOS DE DEEP LEARNING Y APRENDIZAJE AUTOMÁTICO\S02\pytorch_lightning\lightning_logs is not a directory

### 🧪 Paso 5: Evaluación del modelo (opcional)
Puedes extender el método `validation_step` o `test_step` en el modelo para evaluar.
También puedes guardar y cargar el modelo con Lightning.

In [6]:
# Guardar el modelo entrenado
trainer.save_checkpoint("modelo_lightning.ckpt")

# Cargar modelo
modelo_cargado = LitModel.load_from_checkpoint("modelo_lightning.ckpt")
modelo_cargado.eval()

LitModel(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)